In [ ]:
from abc import ABC, abstractmethod
    

class Pair:

    _from_ = None
    _to = None

    def __init__(self, from_, to):
        self._from_ = from_
        self._to = to
    
    def __eq__(self, object):
        return self._from_ == object._from_ and self._to == object._to
    
    def __hash__(self):
        return 0
    

class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, bank, to : str): # -> Money
        pass


    @abstractmethod
    def __add__(self, addend): # -> Expression
        pass


    @abstractmethod
    def __mul__(self, multiplier): # -> Expression
        pass


class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def __mul__(self, multiplier) -> Expression:
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend: Expression) -> Expression:
        return Sum(self, addend)
    

    def reduce(self, bank, to : str): # -> Money
        rate = bank.rate(self._currency, to)
        return Money(self._amount / rate, to)
    

class Sum(Expression):      # implement interface

    augend: Expression | None = None
    addend: Expression | None = None


    def __init__(self, augend: Expression, addend: Expression):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, bank, to: str):
        amount : int = self.augend.reduce(bank, to)._amount \
            + self.addend.reduce(bank, to)._amount
        return Money(amount, to)
    

    def __add__(self, addend: Expression) -> Expression:
        return Sum(self, addend)
    

    def __mul__(self, multiplier) -> Expression:
        return Sum(self.augend * multiplier, self.addend * multiplier)
    

class Bank:


    _rates : dict[Pair, int] = {}


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(self, to)
    

    def rate(self, from_: str, to: str) -> int:
        if from_ == to:
            return 1
        return self._rates[Pair(from_, to)]
    

    def addRate(self, from_: str, to: str, rate: int):
        self._rates[Pair(from_, to)] = rate

# What's next?

Is the code finished? No. There is that nasty duplication between `Sum.plus()` and
`Money.plus()`.

If we made `Expression` a class instead of an interface, we would have a natural
home for the common code.

# Process

Assuming that writing a test is a single step, ***how many changes*** does it take
to compile, run, and refactor? (By change, I mean changing a method or class
***definition***.)

![](../assets/changes-per-refactoring.png)

# Test quality

Here are a couple of widely shared measurements of the tests:

1. ***Statement coverage***: There are several ways to improve coverage. One is to write more tests. Another is to simplifying the logic of the system via refactoring steps.
2. ***Defect insertion***: change the meaning of a line of code and a test should break. You can do this manually, or with a tool such as Jester. For example, Jester will report only one line it is able to change without breaking,
`Pair.hashCode()`. We faked the implementation to just return 0. Returning a
different constant doesn’t actually change the meaning of the program
(one fake number is as good as another), so it isn’t really a defect that has
been inserted.